# Plot composite profile

## Import package

In [6]:
import h5py
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.colors import TwoSlopeNorm
from scipy.stats import percentileofscore

## Load data

In [7]:
REG_PATH = "/work/b11209013/2025_Research/regression/"

# load ERA5 regression
with h5py.File(f"{REG_PATH}IMERG_ERA5.h5", "r") as f:
    q_grp = f["q_composite"]
    t_grp = f["t_composite"]
    
    q_reg = {}
    t_reg = {}

    for key in q_grp.keys():
        q_reg[key] = np.array(q_grp.get(key))
        t_reg[key] = np.array(t_grp.get(key))

# load CloudSat regression
with h5py.File(f"{REG_PATH}IMERG_CLOUDSAT.h5", "r") as f:
    lw_grp = f["lw_composite"]
    sw_grp = f["sw_composite"]
    
    lw_reg = {}
    sw_reg = {}

    for key in lw_grp.keys():
        lw_reg[key] = np.array(lw_grp.get(key))
        sw_reg[key] = np.array(sw_grp.get(key))

# load ERA5 radiation regression
with h5py.File(f"{REG_PATH}IMERG_MOISTRAD.h5", "r") as f:
    q_lw_grp = f["q_lw_composite"]
    q_sw_grp = f["q_sw_composite"]
    t_lw_grp = f["t_lw_composite"]
    t_sw_grp = f["t_sw_composite"]
    
    q_lw_reg = {}
    q_sw_reg = {}
    t_lw_reg = {}
    t_sw_reg = {}

    for key in q_lw_grp.keys():
        q_lw_reg[key] = np.array(q_lw_grp.get(key))
        q_sw_reg[key] = np.array(q_sw_grp.get(key))
        t_lw_reg[key] = np.array(t_lw_grp.get(key))
        t_sw_reg[key] = np.array(t_sw_grp.get(key))

### Compute Cloud Only Component

In [8]:
lw_cloud_reg = {}
sw_cloud_reg = {}

for key in lw_reg.keys():
    lw_cloud_reg[key] = lw_reg[key] - q_lw_reg[key] - t_lw_reg[key]
    sw_cloud_reg[key] = sw_reg[key] - q_sw_reg[key] - t_sw_reg[key]

### Compute vertical profile at convective phase

#### Define functions

In [ ]:
def phase_avg(
    central_idx : np.int64,
    k_ave       : np.float64,
    qlw         : np.ndarray,
    qsw         : np.ndarray
):

    # calculate size of averaging domain
    kernel_lens = np.round(2*np.pi*6.371*1e6*1e-3/(k_ave*4*110))/0.625
    half_kernel = int(kernel_lens // 2)

    # slice of window indexing
    sel_idx = slice(central_idx-half_kernel, central_idx+half_kernel+1)

    # select radiative heating profile
    qlw_comp = qlw[:,sel_idx]
    qsw_comp = qsw[:,sel_idx]

    # average over slected domain
    qlw_mean, qsw_mean = np.nanmean(qlw_comp,axis=-1), np.nanmean(qsw_comp, axis=-1)

    return qlw_mean, qsw_mean

#### Compute for Total Radiation

In [ ]:
lw_tot_prof = {}
sw_tot_prof = {}

for key in lw_reg.keys():
    lw_tot_prof[key], sw_tot_prof[key] = phase_avg(
        int(288),
        k_ave       = 200,
        qlw         = lw_reg[key],
        qsw         = sw_reg[key]
    )

## Plot out composite profile

### Define plot functions

In [10]:
def plot_profile(
        lon, lev, rad_data,
        t_data, q_data,
        cmap, key, type_of_rad
):
    # split key
    split_key = key.split("_")
    hov_type = split_key[0]
    kl = split_key[1]
    kr = split_key[2]

    split_type = type_of_rad.split("_")
    type_of_rad = split_type[0]
    type_of_comp = split_type[1]

    # Calculate levels
    if (type_of_comp == "Total") or (type_of_comp == "Cloud"):
        rad_max   = 0.1
    else:
        rad_max   = 0.0005
    
    t_max     = np.abs(t_data).max() // 0.001
    t_levels  = np.linspace(-t_max*0.001, t_max*0.001+0.001, 15)
    t_levels  = [t_levels[i] for i in range(len(t_levels)) if abs(t_levels[i]) >= 1e-3]
    
    q_max     = np.abs(q_data).max() // 0.001
    q_levels  = np.linspace(-q_max*0.001, q_max*0.001+0.001, 15)
    q_levels  = [q_levels[i] for i in range(len(q_levels)) if abs(q_levels[i]) >= 1e-3]

    fig = plt.figure(figsize=(10.5,6.2))

    pc = plt.pcolormesh(
        lon, lev, rad_data,
        cmap=cmap, norm=TwoSlopeNorm(vmin=-rad_max, vmax=rad_max, vcenter=0),
        alpha=0.5
        )

    ct = plt.contour(lon, lev, t_data, colors='black', linewidths=1.25, levels=t_levels)
    plt.clabel(ct, fmt='%.3f', colors='black', fontsize=8)

    ct = plt.contour(lon, lev, q_data, colors='seagreen', linewidths=1.25, levels=q_levels)
    plt.clabel(ct, fmt='%.3f', colors='seagreen', fontsize=8)

    plt.xticks(fontsize=14)
    plt.yticks(fontsize=14)
    plt.xlim(-100, 100)
    plt.xlabel("Longitude", fontsize=18)
    plt.ylabel("Level (hPa)", fontsize=18)
    plt.title(f"{hov_type.upper()} k={kl}~{kr} {type_of_rad} ({type_of_comp})", fontsize=20)
    plt.gca().invert_yaxis()
    cbar = plt.colorbar(
        pc, label=f"{type_of_rad} Regression Coefficient",
        orientation="horizontal", shrink=0.8, aspect=50
        )
    cbar.ax.tick_params(labelsize=14)
    cbar.ax.xaxis.label.set_size(16)
    plt.tight_layout()
    plt.savefig(f"/home/b11209013/2025_Research/Obs/Figure/{type_of_comp}_Rad/{type_of_rad}/{hov_type}_k={kl}_{kr}.png", dpi=500)
    plt.close()


# loop for total longwave radiation
for key in lw_reg.keys():
    plot_profile(
        np.linspace(-180, 180, 576),
        np.linspace(1000, 100, 37),
        lw_reg[key],
        t_reg[key],
        q_reg[key],
        "RdBu_r",
        key,
        "LW_Total"
    )

# loop for total shortwave radiation
for key in sw_reg.keys():
    plot_profile(
        np.linspace(-180, 180, 576),
        np.linspace(1000, 100, 37),
        sw_reg[key],
        t_reg[key],
        q_reg[key],
        "PiYG_r",
        key,
        "SW_Total"
    )


# loop for moisture longwave radiation
for key in q_lw_reg.keys():
    plot_profile(
        np.linspace(-180, 180, 576),
        np.linspace(1000, 100, 37),
        q_lw_reg[key],
        t_reg[key],
        q_reg[key],
        "RdBu_r",
        key,
        "LW_Moisture"
    )

# loop for moisture shortwave radiation
for key in q_sw_reg.keys():
    plot_profile(
        np.linspace(-180, 180, 576),
        np.linspace(1000, 100, 37),
        q_sw_reg[key],
        t_reg[key],
        q_reg[key],
        "PiYG_r",
        key,
        "SW_Moisture"
    )

# loop for moisture+temperature longwave radiation
for key in q_lw_reg.keys():
    plot_profile(
        np.linspace(-180, 180, 576),
        np.linspace(1000, 100, 37),
        q_lw_reg[key] + t_lw_reg[key],
        t_reg[key],
        q_reg[key],
        "RdBu_r",
        key,
        "LW_Moisture+Temperature"
    )

# loop for moisture+temperature shortwave radiation
for key in q_sw_reg.keys():
    plot_profile(
        np.linspace(-180, 180, 576),
        np.linspace(1000, 100, 37),
        q_sw_reg[key] + t_sw_reg[key],
        t_reg[key],
        q_reg[key],
        "PiYG_r",
        key,
        "SW_Moisture+Temperature"
    )

# loop for cloud longwave radiation
for key in lw_cloud_reg.keys():
    plot_profile(
        np.linspace(-180, 180, 576),
        np.linspace(1000, 100, 37),
        lw_cloud_reg[key],
        t_reg[key],
        q_reg[key],
        "RdBu_r",
        key,
        "LW_Cloud"
    )

# loop for cloud shortwave radiation
for key in sw_cloud_reg.keys():
    plot_profile(
        np.linspace(-180, 180, 576),
        np.linspace(1000, 100, 37),
        sw_cloud_reg[key],
        t_reg[key],
        q_reg[key],
        "PiYG_r",
        key,
        "SW_Cloud"
    )